<a href="https://colab.research.google.com/github/JuliusPain/OG-ZAF/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/JuliusPain/OG-ZAF.git

Cloning into 'OG-ZAF'...
remote: Enumerating objects: 3080, done.
remote: Counting objects: 100% (520/520), done.
remote: Compressing objects: 100% (244/244), done.
remote: Total 3080 (delta 301), reused 299 (delta 276), pack-reused 2560 (from 3)
Receiving objects: 100% (3080/3080), 52.61 MiB | 31.63 MiB/s, done.
Resolving deltas: 100% (1642/1642), done.


In [1]:
%cd /content/OG-ZAF/examples/

/content/OG-ZAF/examples


In [3]:
!pip install ogzaf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/10

In [2]:
# Need to fix references to Calculator, reform json, and substitute new tax
# function call
import multiprocessing
from distributed import Client
import os
import json
import time
import copy
import importlib.resources
import matplotlib.pyplot as plt
from ogcore.parameters import Specifications
from ogcore import output_tables as ot
from ogcore import output_plots as op
from ogcore.execute import runner
from ogcore.utils import safe_read_pickle, param_dump_json

In [3]:
# Use a custom matplotlib style file for plots
plt.style.use("ogcore.OGcorePlots")

In [4]:
# Define parameters to use for multiprocessing
num_workers = min(multiprocessing.cpu_count(),7)
client = Client(n_workers=num_workers, threads_per_worker=1)
print("Number of workers = ", num_workers)

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:39967
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39465'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:38563'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46485'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41273'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42529'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42139'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35477'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:37023 name: 3
INFO:distributed.scheduler:Star

Number of workers =  7


In [5]:
# Directories to save data
CUR_DIR = os.path.dirname("/content/OG-ZAF/examples/")
save_dir = os.path.join(CUR_DIR, "OG-ZAF-MultipleIndustry-example")
base_dir = os.path.join(save_dir, "OUTPUT_BASELINE")
reform_dir = os.path.join(save_dir, "OUTPUT_REFORM")

In [6]:
# Set up baseline parameterization
p = Specifications(
    baseline=True,
    num_workers=num_workers,
    baseline_dir=base_dir,
    output_base=base_dir,
)

In [7]:
# Update parameters for baseline from default json file
with open('/content/OG-ZAF/ogzaf/ogzaf_default_parameters_multisector.json', 'r') as file:
    defaults = json.load(file)
p.update_specifications(defaults)

In [ ]:
# Run model
start_time = time.time()
runner(p, time_path=True, client=client)
print("run time = ", time.time() - start_time)

making dir:  /content/OG-ZAF/examples/OG-ZAF-MultipleIndustry-example/OUTPUT_BASELINE/SS
making dir:  /content/OG-ZAF/examples/OG-ZAF-MultipleIndustry-example/OUTPUT_BASELINE/TPI
In runner, baseline is  True
SS using initial guess factors for r and TR of 1.0 and 1.0 , respectively.
GE loop errors =  [0.004037342801324145, 0.003929447984157383, -0.1612368082815716, 0.4304303672348242, 6.401791351307592, -0.3928230419155311, 0.0, 1.9919543678489813, 0.005227691170410605, 0.0017984318322971733, 0.0021675261961197883, 0.03147709656172726, 0.0511491458754642, 0.12070757202947562, 0.01982610590725882, 0.08167012908180823, 0.06632578408266761]
GE loop errors =  [0.003481449429214599, 0.003286041269915163, -0.13584928916745032, 0.4418380740316237, 6.460820764721142, -0.3879807952251244, 0.0, 2.046155245176257, 0.00747480226506075, 0.030670269528556594, 0.0389335092653531, 0.03146025656580862, 0.05112183157811872, 0.12064322964992989, 0.0198155415644519, 0.08389236505222653, 0.06190464141495643

In [ ]:
# create new Specifications object for reform simulation
p2 = copy.deepcopy(p)
p2.baseline = False
p2.output_base = reform_dir

In [ ]:
# In this scenario, a two percent VAT rate hike and a spending cut are introduced. They are staggared over 4 years
updated_params_ref = {
    "tau_c": [
        [0.15,0.155,0.16,0.165,0.17],
    ],
   "alpha_G": [0.23,0,0.22,0.21,0.20],
}
p2.update_specifications(updated_params_ref)

In [ ]:
# Run model
start_time = time.time()
runner(p2, time_path=True, client=client)
print("run time = ", time.time() - start_time)
client.close()

In [ ]:
#Save some of the results

base_tpi = safe_read_pickle(os.path.join(base_dir, "TPI", "TPI_vars.pkl"))
base_params = safe_read_pickle(os.path.join(base_dir, "model_params.pkl"))
reform_tpi = safe_read_pickle(
    os.path.join(reform_dir, "TPI", "TPI_vars.pkl")
)
reform_params = safe_read_pickle(
    os.path.join(reform_dir, "model_params.pkl")
)
ans = ot.macro_table(
    base_tpi,
    base_params,
    reform_tpi=reform_tpi,
    reform_params=reform_params,
    var_list=["Y", "C", "K", "L", "r", "w"],
    output_type="pct_diff",
    num_years=10,
    start_year=base_params.start_year,
)

In [29]:
import os
print(os.getcwd())

/content/OG-ZAF/ogzaf
